# **Demo Script 2**


This script is the testing portion of the trained model analysis. Specifically this script loads in the test fNIRS and fMRI data, as well as the trained model (from demo 1). The trained model is then run on the testing data and the model performance is found.


In [1]:
import scipy.io as sio
import pickle

import numpy as np
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

## **Test the model on all of Run 2 Data**

In [2]:
## Test a model on each subject in Run 2

# Load fNIRS test data
fnirs_file2 = '../data/fnirs/clip2_noUncertain_zhbo_TxRxS.mat'
test_fNIRS = sio.loadmat(fnirs_file2)['zhbo_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
# get shape
print(f'test_fNIRS shape is {test_fNIRS.shape}')

# let's impute missing channels with mean of all other channels
test_fNIRS = np.array(
    [
        np.apply_along_axis(lambda x: np.nan_to_num(x, nan=np.nanmean(x)), axis=1, arr=curr_region) \
            for curr_region in test_fNIRS.transpose((2, 1, 0))  # transpose to shape: RxTxS
    ]
).transpose((2, 1, 0))

# load fMRI test data
fmri_file2 = '../data/fmri/clip2_undenoised_withcartoon_fillmean_TxRxS.mat'
test_fMRI = sio.loadmat(fmri_file2)['bold_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
test_fMRI_mean = np.mean(test_fMRI, axis=0)  # already imputed, so we use np.mean instead of np.nanmean
# get shape
print(f'test_fMRI shape is {test_fMRI.shape}')

# Load the model
file_path = '../models/sherlock_run1model.pickle'
with open(file_path, 'rb') as f:
   X_pca, y_pca, lr_reg = pickle.load(f)

# get pc loadings for X and y
X_pca_loadings = X_pca.components_
y_pca_loadings = y_pca.components_

print("Model loaded...")


test_fNIRS shape is (29, 1030, 20)
test_fMRI shape is (17, 1030, 122)
Model loaded...


In [3]:
model_performance = []
model_predicted_tc = []

for subj in range(test_fNIRS.shape[0]): # range(1): # range(test_fNIRS.shape[0]):
  curr_test_fNIRS = test_fNIRS[subj,:,:]
  fNIRS_test_pc = X_pca.transform(curr_test_fNIRS)
  model_predicted_PC = lr_reg.predict(X=fNIRS_test_pc)
  model_predicted_tc = y_pca.inverse_transform(model_predicted_PC)

  corr_per_roi = np.array([pearsonr(model_predicted_tc[:, i], test_fMRI_mean[:, i])[0] for i in range(test_fMRI_mean.shape[1])])
  model_performance.append(corr_per_roi)

  print(f"S{subj +1} median r = {np.median(corr_per_roi).round(4)}")

model_performance = np.array(model_performance)

S1 median r = 0.118
S2 median r = 0.1139
S3 median r = 0.008
S4 median r = 0.053
S5 median r = 0.0486
S6 median r = 0.0546
S7 median r = 0.0787
S8 median r = 0.0449
S9 median r = 0.0666
S10 median r = 0.0322
S11 median r = 0.0532
S12 median r = 0.0423
S13 median r = 0.0761
S14 median r = 0.1052
S15 median r = 0.1428
S16 median r = 0.0433
S17 median r = 0.0582
S18 median r = 0.0032
S19 median r = 0.0085
S20 median r = 0.0383
S21 median r = 0.0723
S22 median r = 0.0194
S23 median r = 0.0963
S24 median r = -0.0123
S25 median r = 0.0543
S26 median r = 0.036
S27 median r = 0.0772
S28 median r = 0.0469
S29 median r = 0.1393


In [4]:
median_model_performance = np.median(model_performance, axis=0)

In [5]:
#Saving the model data
# Create a dictionary to store the variables
data = {'model_predicted_tc': model_predicted_tc,
        'median_model_performance': median_model_performance,
        'curr_test_fNIRS': curr_test_fNIRS,
        'fNIRS_test_pc': fNIRS_test_pc,
        'model_predicted_PC': model_predicted_PC,
        'model_performance': model_performance}

#Save the mat file
file_path = '../models/model_performance.mat'
sio.savemat(file_path, data)